In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('C:\\Users\\user\\Desktop\\Python\\Lakshmi\\severeinjury1.csv',encoding='cp1252')
                   

In [3]:
import nltk

In [4]:
from nltk.util import ngrams

In [5]:
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.snowball import SnowballStemmer
stemming = SnowballStemmer("english")
data["narrative"]= data["title.new"]+data["summary.new"]

In [6]:
def identify_tokens(row):
    summary_title = row["narrative"]
    tokens = nltk.word_tokenize(summary_title)
    token_words = ngrams(tokens,3)
    tok = [" ".join(t) for t in token_words]
    return tok

In [7]:
data['words']= data.apply(identify_tokens,axis=1)
print(data['words'])

0      [two workers are, workers are struck, are stru...
1      [foreman is fatally, is fatally crushed, fatal...
2      [employee suffers abdominal, suffers abdominal...
3      [employee 's body, 's body is, body is caught,...
4      [employee is punctured, is punctured in, punct...
                             ...                        
995    [two employees are, employees are killed, are ...
996    [employee suffered face, suffered face injury,...
997    [machine operator is, operator is injured, is ...
998    [two employees are, employees are killed, are ...
999    [employee fall from, fall from roof, from roof...
Name: words, Length: 1000, dtype: object


In [8]:
def stem_list(row):
    my_list = row['words']
    stemmed_list = [stemming.stem(word) for word in my_list]
    return(stemmed_list)

In [9]:
data['stemmed_words'] = data.apply(stem_list, axis=1)
data['stemmed_words']

0      [two workers ar, workers are struck, are struc...
1      [foreman is fat, is fatally crush, fatally cru...
2      [employee suffers abdomin, suffers abdominal f...
3      [employee 's bodi, s body i, body is caught, i...
4      [employee is punctur, is punctured in, punctur...
                             ...                        
995    [two employees ar, employees are kil, are kill...
996    [employee suffered fac, suffered face injuri, ...
997    [machine operator i, operator is injur, is inj...
998    [two employees ar, employees are kil, are kill...
999    [employee fall from, fall from roof, from roof...
Name: stemmed_words, Length: 1000, dtype: object

In [10]:
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))                  


In [11]:
def remove_stops(row):
    my_list = row['stemmed_words']
    meaningful_words = [w for w in my_list if not w in stops]
    return (meaningful_words)

In [12]:
data['stem_meaningful'] = data.apply(remove_stops, axis=1)
data['stem_meaningful']

0      [two workers ar, workers are struck, are struc...
1      [foreman is fat, is fatally crush, fatally cru...
2      [employee suffers abdomin, suffers abdominal f...
3      [employee 's bodi, s body i, body is caught, i...
4      [employee is punctur, is punctured in, punctur...
                             ...                        
995    [two employees ar, employees are kil, are kill...
996    [employee suffered fac, suffered face injuri, ...
997    [machine operator i, operator is injur, is inj...
998    [two employees ar, employees are kil, are kill...
999    [employee fall from, fall from roof, from roof...
Name: stem_meaningful, Length: 1000, dtype: object

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import math
data['stem_meaningful'] = ["  ".join(review) for review in data['stem_meaningful'].values]
data['stem_meaningful']

0      two workers ar  workers are struck  are struck...
1      foreman is fat  is fatally crush  fatally crus...
2      employee suffers abdomin  suffers abdominal fr...
3      employee 's bodi  s body i  body is caught  is...
4      employee is punctur  is punctured in  puncture...
                             ...                        
995    two employees ar  employees are kil  are kille...
996    employee suffered fac  suffered face injuri  f...
997    machine operator i  operator is injur  is inju...
998    two employees ar  employees are kil  are kille...
999    employee fall from  fall from roof  from roof ...
Name: stem_meaningful, Length: 1000, dtype: object

In [14]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
v=TfidfVectorizer()
x=v.fit_transform(data['stem_meaningful'])
first_vector_v = x[0]

In [15]:
df = pd.DataFrame(first_vector_v.T.todense(),index = v.get_feature_names(),columns = ["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
lane,0.263434
vehicle,0.230604
highway,0.224491
turn,0.215269
templar,0.187255
...,...
expand,0.000000
exiting,0.000000
exited,0.000000
exit,0.000000


In [16]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes,svm
from sklearn.metrics import accuracy_score

In [17]:
import numpy as np
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data['summary.new'],data['Tagged2'],test_size=0.2)

In [18]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
Train_X_Tfidf = v.transform(Train_X)
Test_X_Tfidf = v.transform(Test_X)


In [19]:
print(v.vocabulary_)
print(Train_X_Tfidf)

{'two': 8286, 'workers': 8843, 'ar': 764, 'are': 777, 'struck': 7604, 'bi': 1093, 'by': 1357, 'motor': 5099, 'vehicl': 8552, 'vehicle': 8553, 'and': 670, 'on': 5370, 'one': 5376, 'is': 4265, 'kil': 4402, 'killed': 4404, 'august': 894, '27': 178, '2013': 143, 'employe': 2785, 'employees': 2789, 'of': 5348, 'templar': 7908, 'inc': 4029, 'construct': 1927, 'construction': 1930, 'compani': 1806, 'company': 1808, 'speci': 7315, 'specializing': 7320, 'in': 4018, 'fib': 3178, 'fiber': 3179, 'opt': 5409, 'optic': 5410, 'instal': 4162, 'installation': 4165, 'servic': 6925, 'services': 6928, 'wer': 8737, 'were': 8738, 'work': 8838, 'working': 8844, 'along': 607, 'highway': 3817, 'th': 7943, 'the': 7950, 'spe': 7313, 'speed': 7334, 'limit': 4646, 'wa': 8648, 'was': 8680, 'post': 5882, 'posted': 5883, 'at': 854, '55': 284, 'mil': 4972, 'miles': 4977, 'per': 5655, 'hour': 3896, 'employee': 2787, 'mark': 4848, 'marking': 4850, 'loc': 4684, 'location': 4692, 'an': 656, 'underground': 8343, 'lin': 464

In [20]:
SVM = svm.SVC(C=0.80,kernel='linear')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
target_names = ['Caught in/between objects','Collapse of object','Electrocution','Exposure to chemical substances','Exposure to extreme temperatures','Falls','Fires and explosions','Struck by moving objects','Struck by falling objects','Traffic','Others']


In [21]:
print(confusion_matrix(Test_Y,predictions_SVM))

[[ 3  3  0  0  0  0  0  0  0  7  1]
 [ 0 34  0  0  0  8  0  0  0  0  0]
 [ 1  1 24  0  0  1  0  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  3  0]
 [ 0  1  0  0  0  1  0  0  0  0  0]
 [ 0 15  0  0  0 46  0  0  0  0  0]
 [ 0  1  0  0  0  0  7  0  0  2  0]
 [ 0  2  1  0  0  0  0  0  0  2  0]
 [ 0  6  0  0  0  1  0  0  0  0  0]
 [ 0 11  0  0  0  1  1  0  0  6  0]
 [ 0  3  0  0  0  0  0  0  0  1  2]]


In [26]:
print(classification_report(Test_Y,predictions_SVM,target_names = target_names))

                                  precision    recall  f1-score   support

       Caught in/between objects       1.00      0.36      0.53        11
              Collapse of object       0.55      0.75      0.64        55
                   Electrocution       1.00      0.89      0.94        19
 Exposure to chemical substances       0.00      0.00      0.00         4
Exposure to extreme temperatures       0.00      0.00      0.00         3
                           Falls       0.61      0.88      0.72        43
            Fires and explosions       1.00      0.54      0.70        13
        Struck by moving objects       0.00      0.00      0.00         8
       Struck by falling objects       0.00      0.00      0.00         9
                         Traffic       0.55      0.57      0.56        28
                          Others       0.71      0.71      0.71         7

                        accuracy                           0.64       200
                       macro avg    

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
SVM_accuracy = accuracy_score(Test_Y, predictions_SVM)
print(accuracy_score(Test_Y, predictions_SVM))

0.58


In [22]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
predictions_NB = Naive.predict(Test_X_Tfidf)
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [23]:
print(confusion_matrix(Test_Y,predictions_NB))

[[ 0  3  0  0  0  8  0  0  0  0  0]
 [ 0 21  0  0  0 20  0  0  0  0  0]
 [ 0  3  2  0  0 13  0  0  0  0  0]
 [ 0  2  0  0  0  4  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0]
 [ 0  3  0  0  0 49  0  0  0  0  0]
 [ 0  2  0  0  0 15  0  0  0  0  0]
 [ 0  1  0  0  0  9  0  0  0  0  0]
 [ 0  5  0  0  0  3  0  0  0  0  0]
 [ 0 12  0  0  0  9  0  0  0  1  0]
 [ 0  5  0  0  0  7  0  0  0  0  0]]


In [30]:
print(classification_report(Test_Y,predictions_NB,target_names = target_names))

                                  precision    recall  f1-score   support

       Caught in/between objects       0.00      0.00      0.00        11
              Collapse of object       0.53      0.16      0.25        55
                   Electrocution       1.00      0.11      0.19        19
 Exposure to chemical substances       0.00      0.00      0.00         4
Exposure to extreme temperatures       0.00      0.00      0.00         3
                           Falls       0.24      1.00      0.39        43
            Fires and explosions       0.00      0.00      0.00        13
        Struck by moving objects       0.00      0.00      0.00         8
       Struck by falling objects       0.00      0.00      0.00         9
                         Traffic       1.00      0.04      0.07        28
                          Others       0.00      0.00      0.00         7

                        accuracy                           0.28       200
                       macro avg    

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
Naive_Accuracy = accuracy_score(Test_Y, predictions_NB)
print(accuracy_score(Test_Y, predictions_NB))

0.365


In [25]:
from sklearn.tree import DecisionTreeClassifier
clf3 = DecisionTreeClassifier()

In [26]:
clf3.fit(Train_X_Tfidf,Train_Y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [27]:
y_pred = clf3.predict((Test_X_Tfidf))
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [41]:
print(confusion_matrix(Test_Y,y_pred))
print(classification_report(Test_Y,y_pred,target_names = target_names))

[[ 6  2  0  0  0  0  0  1  1  2  2]
 [ 1 15  1  2  2  7  1  3  3  4  3]
 [ 0  0 21  0  0  2  3  0  0  1  0]
 [ 0  0  1  1  0  0  2  1  0  1  1]
 [ 0  1  0  0  0  1  0  0  0  0  0]
 [ 0 18  0  0  0 36  0  2  3  2  0]
 [ 0  1  0  0  1  1  5  1  1  0  0]
 [ 0  1  0  0  1  1  0  2  0  0  0]
 [ 0  2  0  0  0  1  0  0  1  2  1]
 [ 1  3  0  0  0  0  1  1  0 13  0]
 [ 0  1  0  0  0  1  0  0  0  1  3]]
                                  precision    recall  f1-score   support

       Caught in/between objects       0.75      0.43      0.55        14
              Collapse of object       0.34      0.36      0.35        42
                   Electrocution       0.91      0.78      0.84        27
 Exposure to chemical substances       0.33      0.14      0.20         7
Exposure to extreme temperatures       0.00      0.00      0.00         2
                           Falls       0.72      0.59      0.65        61
            Fires and explosions       0.42      0.50      0.45        10
        St

In [28]:
DecisionTree_Accuracy = accuracy_score(Test_Y, y_pred)
print(accuracy_score(Test_Y, y_pred))

0.495


In [29]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=25)
classifier.fit(Train_X_Tfidf,Train_Y)
y_pred = classifier.predict((Test_X_Tfidf))
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [44]:
print(confusion_matrix(Test_Y,y_pred))
print(classification_report(Test_Y,y_pred,target_names = target_names))

[[ 5  4  0  0  0  1  0  0  0  3  1]
 [ 1 25  0  0  0 14  0  0  1  1  0]
 [ 1  2 23  0  0  1  0  0  0  0  0]
 [ 1  2  0  1  0  1  0  0  0  0  2]
 [ 0  0  0  0  1  1  0  0  0  0  0]
 [ 0  8  0  0  0 53  0  0  0  0  0]
 [ 0  2  0  0  1  3  4  0  0  0  0]
 [ 0  1  0  0  0  4  0  0  0  0  0]
 [ 0  2  0  0  0  5  0  0  0  0  0]
 [ 1  8  0  0  0  3  0  0  0  6  1]
 [ 0  2  0  0  0  0  0  0  0  1  3]]
                                  precision    recall  f1-score   support

       Caught in/between objects       0.56      0.36      0.43        14
              Collapse of object       0.45      0.60      0.51        42
                   Electrocution       1.00      0.85      0.92        27
 Exposure to chemical substances       1.00      0.14      0.25         7
Exposure to extreme temperatures       0.50      0.50      0.50         2
                           Falls       0.62      0.87      0.72        61
            Fires and explosions       1.00      0.40      0.57        10
        St

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
KNeighbors_Accuracy = accuracy_score(Test_Y, y_pred)
print(accuracy_score(Test_Y, y_pred))

0.53


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
import pickle
from sklearn.linear_model import LogisticRegression
X= data['summary.new']
Y = data['Tagged2']
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.2)
pipeline = Pipeline([('vect', v),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', LogisticRegression(random_state=0))])
model = pipeline.fit(X_train, y_train)
with open('LogisticRegression.pickle', 'wb') as f:
    pickle.dump(model, f)
    ytest = np.array(y_test)

In [32]:
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))

                                  precision    recall  f1-score   support

       caught in/between objects       1.00      0.25      0.40        12
              collapse of object       0.33      0.74      0.45        35
                   electrocution       1.00      0.83      0.91        24
 exposure to chemical substances       0.00      0.00      0.00         7
exposure to extreme temperatures       0.00      0.00      0.00         4
                           falls       0.66      0.87      0.75        54
             fires and explosion       0.00      0.00      0.00         6
                          others       0.00      0.00      0.00         6
        struck by falling object       0.00      0.00      0.00         5
        struck by moving objects       0.48      0.33      0.39        30
                         traffic       0.80      0.24      0.36        17

                        accuracy                           0.55       200
                       macro avg    

C:\Users\user\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:
LR_Accuracy = accuracy_score(Test_Y, y_pred)
print(accuracy_score(Test_Y, y_pred))

0.53


In [34]:
import random
best=-100000
populations = [random.randint(0.0,1.0) for x in range(6)] 
print(type(populations))
parents=[]
new_populations = []
print(populations)

<class 'list'>
[1, 0, 0, 0, 1, 1]


In [35]:
equation_inputs = [SVM_accuracy,DecisionTree_Accuracy,Naive_Accuracy,KNeighbors_Accuracy,LR_Accuracy]

In [36]:
equation_inputs = [0.58,0.365,0.495,0.53,0.53]

In [37]:
num_weights = 5

In [38]:
import numpy
sol_per_pop = 8

In [39]:
pop_size = (sol_per_pop,num_weights)

In [40]:
new_population = numpy.random.uniform(low=-0.1, high=0.5, size=pop_size)
import numpy


In [41]:
import ga

In [42]:
sol_per_pop = 8
num_parents_mating = 4

In [43]:
pop_size = (sol_per_pop,num_weights) 

In [44]:
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)
print(new_population)


[[ 0.26190889 -0.08099718  0.33867578 -3.27429307 -2.92039411]
 [ 2.71252192 -2.33950075 -2.92790901 -3.84429166 -0.69036867]
 [ 2.77024438  3.34905237 -3.79914875  0.97917997  3.31130822]
 [-3.27742739 -0.97821286 -0.87286364  0.15258908  0.10702933]
 [-2.00463435  0.11101269 -2.81433439 -2.98546448 -2.33076642]
 [-0.84209476  3.64835035 -3.88286422 -2.47597946  1.84769031]
 [-2.89497136 -0.31476756 -3.54969318 -0.78139801  3.65503868]
 [ 1.82867544  3.59518346 -0.90465286 -2.20637151 -1.62677717]]


In [45]:
num_generations = 5
for generation in range(num_generations):
    print("Generation : ", generation)

Generation :  0
Generation :  1
Generation :  2
Generation :  3
Generation :  4


In [51]:
from geneticalgorithm import geneticalgorithm as ga

ModuleNotFoundError: No module named 'geneticalgorithm'

In [46]:
def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

In [52]:
fitness = ga.cal_pop_fitness(equation_inputs, new_population)

AttributeError: module 'ga' has no attribute 'cal_pop_fitness'

In [81]:
 parents = ga.select_mating_pool(new_population, fitness, 
                                      num_parents_mating)


AttributeError: module 'ga' has no attribute 'select_mating_pool'

In [78]:
def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

In [82]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring


In [83]:
def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

In [86]:
fitness = ga.cal_pop_fitness(equation_inputs, new_population)

AttributeError: module 'ga' has no attribute 'cal_pop_fitness'

In [102]:
import ga
num_generations = 5

num_parents_mating = 4
for generation in range(num_generations):
    # Measuring the fitness of each chromosome in the population.
    fitness = ga.cal_pop_fitness(equation_inputs, new_population)


AttributeError: module 'ga' has no attribute 'cal_pop_fitness'

In [98]:
pip install GA

Note: you may need to restart the kernel to use updated packages.


In [112]:
 parents = ga.select_mating_pool(new_population, fitness, 
                                      num_parents_mating)

AttributeError: module 'ga' has no attribute 'select_mating_pool'

In [113]:
pip install pygad

Note: you may need to restart the kernel to use updated packages.


In [125]:
import pygad

In [129]:
import ga

In [131]:
pip install ga

Note: you may need to restart the kernel to use updated packages.


In [126]:
def cal_pop_fitness(equation_inputs, pop):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function caulcuates the sum of products between each input and its corresponding weight.
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

In [130]:
fitness = ga.cal_pop_fitness(equation_inputs, new_population)

AttributeError: module 'ga' has no attribute 'cal_pop_fitness'